In [ ]:
import matplotlib.colors as mcolors
import pandas as pd
from google.colab import drive
import matplotlib.font_manager as fm
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 한글폰트 설치

In [ ]:
# 나눔바른고딕 글꼴 파일 경로 지정
current_directory = "/content/drive/MyDrive/Colab Notebooks"
font_path = os.path.join(current_directory, "NanumBarunGothic.ttf")

#글꼴이 있는지 확인
if os.path.exists(font_path):
    #폰트 추가
    fm.fontManager.addfont(font_path)

    # 캐시를 새로 고치지 않고 직접 폰트 설정
    font_prop = fm.FontProperties(fname=font_path)
    plt.rc("font", family=font_prop.get_name())
    plt.rcParams["axes.unicode_minus"] = False # 마이너스 부호 출력 설정

# 1. 데이터 불러오기 및 데이터 확인

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KOREA IT/m3_분석라이브러리/crawling/Interpark_books_dataset.csv')

df.info()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/KOREA IT/m3_분석라이브러리/crawling/Interpark_books_dataset.csv'

# 전처리

In [ ]:
# 결측치 확인
missing_counts = df.isnull().sum()
print(missing_counts)

In [ ]:
# 통계 확인
df.describe()

In [ ]:
df.head()

# 2. groupby 카테고리 count pie chart , box plot

#### 2-1. 독서 트렌드 파악에 유용하게 활용될 수 없는 컬럼 삭제 ( ex. 학습서 .. )


In [ ]:
print(df.Category.value_counts())

In [ ]:
df.Category.value_counts()

filtered_df = df[~df['Category'].isin(['초중고생학습서', '국어/외국어/사전', '자격서/수험서', '전공도서/대학교재', '잡지'])]

# 필터링된 데이터셋 확인
print(filtered_df['Category'].value_counts())


저희의 특정 칼럼들을 지울 필요가 있다고 생각했습니다.

- 목적과 관련성: 초중고생 학습서, 자격서/수험서, 전공도서/대학교재와 같은 책들은 주로 교육적인 목적이나 전문 지식 습득을 목적으로 합니다. 이들은 독서량 증진을 위한 일반적인 독서 활동과는 목적이 다를 수 있습니다.

- 대상 독자의 특성: 이러한 유형의 책들은 주로 특정 시험이나 학습 목표를 가진 독자들에 의해 구매됩니다. 따라서 이들 데이터는 일반 독서 패턴을 분석하는 데 있어서 다소 왜곡 요소가 될 수 있습니다.

-  분석의 목적 설정: 독서량 증진을 목적으로 하는 경우, 일반 독서 활동에 집중된 데이터가 필요합니다. 이를 통해 독서 문화의 변화와 관련된 인사이트를 정확하게 도출할 수 있습니다.

위와 같은 이유로 초중고생 학습서, 자격서/수험서, 전공도서/대학교재와 같은 데이터를 분석에서 제외하여 일반 독서 활동과 관련된 인사이트를 더욱 정확하게 얻고자 합니다.

#### 2-2. 데이터가 적은 비슷한 카테고리끼리 결합
 - 날짜(Date) 칼럼 생성 (Year + Month)
 - Sales 정수형 변환
 - Age 칼럼 이름수정

In [ ]:
# 여행, 취미/레저를 예술/대중문화에 합치기
filtered_df['Category'] = filtered_df['Category'].replace(['여행', '취미/레저', '예술/대중문화'], '예술/대중문화/취미')

# 유아동전집 및 유아, 아동을 유아동으로 합치기
filtered_df['Category'] = filtered_df['Category'].replace(['유아동전집', '유아', '아동'], '유아동')

# 일본도서를 외국도서로 합치기
filtered_df['Category'] = filtered_df['Category'].replace('일본도서', '외국도서')

# 역사와 사회과학을 사회과학 / 역사로 합치기
filtered_df['Category'] = filtered_df['Category'].replace(['역사', '사회과학'], '사회과학 / 역사')

# 건강/미용을 가정과 생활로 합치기
filtered_df['Category'] = filtered_df['Category'].replace('건강/미용', '가정과 생활')

# 날짜 칼럼 생성
df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(DAY=1))
filtered_df['Date'] = pd.to_datetime(df[['Year', 'Month']].assign(DAY=1))

# 데이터셋 Sale 칼럼 Type 정수형 변환
df['Sales'] = df['Sales'].str.extract('(\d+)').astype(int)
filtered_df['Sales'] = filtered_df['Sales'].str.extract('(\d+)').astype(int)

# 연령대 변환
age_mapping = {1: '10대', 2: '20대', 3: '30대', 4: '40대', 5: '50대', 6: '60대'}
df['Age'] = df['Age'].replace(age_mapping)
filtered_df['Age'] = filtered_df['Age'].replace(age_mapping)

# 합쳐진 결과 확인
print(filtered_df['Category'].value_counts())

In [ ]:
filtered_df.info()

In [ ]:
filtered_df.head()

# 3. 시간의 흐름에 따른 독서량 추세 (꺾은 선 그래프 (2x1) subplot)
 - 학습서 전처리 전/후 subplot    ==>   10대, 30대 독서량 변화.
 - 코로나 초창기 17~18 년도 독서량 급격히 증가.

####3-1 기간 경과에 따른 독서량 변화 (기준: 연도)

In [ ]:
date_sales_age = filtered_df.groupby(['Year', 'Age'])['Sales'].sum().reset_index()

# 기간별 독서량 변화
plt.figure(figsize=(16, 8))
sns.lineplot(data=date_sales_age, x='Year', y='Sales', hue='Age', style='Age', markers=True, palette='tab10',
             linewidth=2.5, dashes=False)

plt.title('기간 & 연령대별 독서량 변화', fontsize=20, weight='bold')
plt.xlabel('기간', fontsize=16, loc='right', weight='bold')
plt.ylabel('독서량', fontsize=16, rotation=0, loc='top', weight='bold')
plt.legend(title='연령대', title_fontsize=14, loc='upper right', fontsize=12, frameon=True, shadow=True)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(date_sales_age['Year'].unique(), rotation=45)
plt.tight_layout()
plt.show()

####3-2 연령대별 독서량 변화

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 연도별 책 판매량을 연령대별로 그룹화
dately_counts = df.groupby(['Date', 'Age'])['Sales'].sum().reset_index()
filtered_dately_counts = filtered_df.groupby(['Date', 'Age'])['Sales'].sum().reset_index()

# 전처리 전 데이터의 연령대별 총 도서구매량 계산
age_sales_before = df.groupby('Age')['Sales'].sum().reset_index()
age_sales_before['Status'] = '전처리 전'

# 전처리 후 데이터의 연령대별 총 도서구매량 계산
age_sales_after = filtered_dately_counts.groupby('Age')['Sales'].sum().reset_index()
age_sales_after['Status'] = '전처리 후'

# 전처리 전과 후 데이터를 하나의 DataFrame으로 결합
combined_sales = pd.concat([age_sales_before, age_sales_after])

# 라인 그래프 그리기
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(16, 16))

# 첫 번째 그래프 (전처리 전 데이터)
for age, data in dately_counts.groupby('Age'):
    if age == '10대':
        sns.lineplot(data=data, x='Date', y='Sales', ax=ax1, label=f'{age}', linewidth=4, color='magenta')
    elif age == '30대':
        sns.lineplot(data=data, x='Date', y='Sales', ax=ax1, label=f'{age}', linewidth=2, color='red')
    else:
        sns.lineplot(data=data, x='Date', y='Sales', ax=ax1, label=f'{age}', linewidth=1)

ax1.set_title('시간 흐름에 따른 연령대별 독서량 변동 그래프 (전처리 전)', fontdict={'fontsize': 24})
ax1.set_xlabel('날짜', fontsize=16, loc='right')
ax1.set_ylabel('독서량', fontsize=16, rotation=0, loc='top')
ax1.legend(title='연령대', title_fontsize=16, loc='upper right', fontsize=16)
ax1.set_xlim(dately_counts['Date'].min(), dately_counts['Date'].max())
ax1.set_ylim(0, dately_counts['Sales'].max() * 1.1)

# 두 번째 그래프 (전처리 후 데이터)
for age, data in filtered_dately_counts.groupby('Age'):
    if age == '10대':
        sns.lineplot(data=data, x='Date', y='Sales', ax=ax2, label=f'{age}', linewidth=4, color='magenta')
    elif age == '30대':
        sns.lineplot(data=data, x='Date', y='Sales', ax=ax2, label=f'{age}', linewidth=4, color='red')
    else:
        sns.lineplot(data=data, x='Date', y='Sales', ax=ax2, label=f'{age}', linewidth=1)

ax2.set_title('시간 흐름에 따른 연령대별 독서량 변동 그래프 (전처리 후)', fontdict={'fontsize': 24})
ax2.set_xlabel('날짜', fontsize=16, loc='right')
ax2.set_ylabel('독서량', fontsize=16, rotation=0, loc='top')
ax2.legend(title='연령대', title_fontsize=16, loc='upper right', fontsize=16)
ax2.set_xlim(filtered_dately_counts['Date'].min(), filtered_dately_counts['Date'].max())
ax2.set_ylim(0, filtered_dately_counts['Sales'].max() * 1.1)

# 세 번째 그래프 (전처리 전과 후 연령대별 도서구매량 비교)
custom_palette = ['black', 'red']
sns.barplot(data=combined_sales, x='Age', y='Sales', hue='Status', palette=custom_palette)

ax3.set_title('전처리 전 / 후 도서구매량 비교', fontsize=24)
ax3.set_xlabel('연령대', fontsize=16)
ax3.set_ylabel('독서량', fontsize=16, rotation=0, loc='top')
ax3.legend(title='비교', title_fontsize=16, fontsize=16)

# 그래프 간 간격 조정
plt.tight_layout()
plt.show()

# 그래프 간 간격 조정
plt.tight_layout()
plt.show()

# 연령대 독서량 변화 시각화 : 전처리 적용 전/후 비교 결과
### 연령층 중 10대, 30대, 40대 유의미한 변화 확인 가능
 - 10대
  - 전처리 전 : 연초에 집중된 도서구매량 분포를 확인할 수 있습니다. 그로인해 학기가 시작하는 시기에 집중되있음을 예측할 수 있습니다.
  - 전처리 후 : 학습도서를 전처리한 결과 학기초에 집중된 도서구매량이 급락한 모습을 그래프를 통해 확인할 수 있습니다. 그로인해 10대의 독서량이 순위가 하락한 모습 또한 확인할 수 있습니다.
 - 30대
  - 교육, 전문지식등 목적에 부합되지 않는 도서를 전처리한 결과 가장 독서를 위한 도서 구매 활동에 가장 활발한 연령대가 30대 라는 것을 확인할 수 있었습니다. 전처리를 진행하여 목표에 부합되지 않는 도서 카테고리를 제거하며 모든 연령대가 도서 구매량이 감소하는 상황 속에서 감소량이 가장 적으며 상대적으로 그래프 곡선이 상향된 것 을 확인할 수 있습니다.

 - 40대
  - 40대 연령층은 전처리 전 가장 많은 도서구매량을 확인할 수 있으나, 전처리 후 가장 큰 감소량을 보였습니다. 이로인해 40대가 구매하는 도서종류가 전처리한 학습도서가 의외로 20대보다 많은 비중을 차지한다는 사실을 확인할 수 있었습니다.

In [ ]:
filtered_df.info()

# 시간 흐름에 따른 도서구매량(기준:년도)
 - 사회적인 이슈가 되고있는 독서량 감소를 확인하기 위해 전체적인 동향을 확인하기 위해 연도로 시각화한 결과 2017년부터 2024년의 기간동안 전반적으로 우하향 곡선을 그리며 독서량이 떨어지고 있다는걸 확인할 수 있습니다.
 다만 2019~2021년도에 하향 곡선의 기울기가 떨어지거나, 반대로 상승하는걸 확인할 수 있습니다. 이는 당시 세계적인 빅이슈였던 코로나로 인한 대책으로 '사회적 거리두기'로 인한 일상 변화에 따른 낙수효과로 분석할 수 있습니다.

# 4. [10대 20대 30대 40대 50대 60대] 인기있는 카테고리(17.01 - 24.06) pie chart (6x2) subplot


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


age_groups = df['Age'].unique()

# 진한 색상의 커스텀 리스트
custom_colors = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2',
    '#D500B7', '#bcbd22', '#17becf', '#393b79', '#637939', '#8c6d31', '#843c39',
    '#7b4173', '#5254a3', '#6b6ecf', '#9c9ede', '#cedb9c', '#e7ba52'
]
# 모든 카테고리 추출 및 색상 맵 생성
all_categories = df['Category'].unique()
color_map = dict(zip(all_categories, custom_colors[:len(all_categories)]))
color_map['그 외 장르'] = '#808080'  # 그 외 장르는 회색으로 설정

fig, axes = plt.subplots(2, 3, figsize=(20, 14))

for i, age_label in enumerate(age_groups):
    row, col = divmod(i, 3)

    # 해당 연령대의 카테고리 선호도 계산
    preferences = df[df['Age'] == age_label]['Category'].value_counts()

    if preferences.empty:
        axes[row, col].set_title(f'{age_label} Preferences (No Data)', size=14)
        continue

    # 상위 8개 카테고리 선택 및 나머지를 '그 외 장르'로 묶기
    top_8 = preferences.nlargest(8)
    others = pd.Series({'그 외 장르': preferences.sum() - top_8.sum()})
    plot_data = pd.concat([top_8, others])

    # 색상 리스트 생성
    colors = [color_map.get(cat, '#808080') for cat in plot_data.index]

    # 파이 차트 그리기
    wedges, texts, autotexts = axes[row, col].pie(plot_data, labels=plot_data.index, autopct='%1.1f%%',
                                                  startangle=90, colors=colors)

    # 레이블 텍스트 크기 조정
    plt.setp(texts, size=10)
    plt.setp(autotexts, size=10, weight="bold")

    axes[row, col].set_title(f'{age_label}', size=18)

plt.suptitle('연령대별 도서 장르 선호도', size=24)
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


age_groups = filtered_df['Age'].unique()

# 진한 색상의 커스텀 리스트
custom_colors = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2',
    '#D500B7', '#bcbd22', '#17becf', '#393b79', '#637939', '#8c6d31', '#843c39',
    '#7b4173', '#5254a3', '#6b6ecf', '#9c9ede', '#cedb9c', '#e7ba52'
]
# 모든 카테고리 추출 및 색상 맵 생성
all_categories = filtered_df['Category'].unique()
color_map = dict(zip(all_categories, custom_colors[:len(all_categories)]))
color_map['그 외 장르'] = '#808080'  # 그 외 장르는 회색으로 설정

fig, axes = plt.subplots(2, 3, figsize=(20, 14))

for i, age_label in enumerate(age_groups):
    row, col = divmod(i, 3)

    # 해당 연령대의 카테고리 선호도 계산
    preferences = filtered_df[filtered_df['Age'] == age_label].groupby('Category')['Sales'].sum()

    if preferences.empty:
        axes[row, col].set_title(f'{age_label} Preferences (No Data)', size=14)
        continue

    # 상위 8개 카테고리 선택 및 나머지를 '그 외 장르'로 묶기
    top_7 = preferences.nlargest(7)
    others = pd.Series({'그 외 장르': preferences.sum() - top_7.sum()})
    plot_data = pd.concat([top_7, others])

    # 색상 리스트 생성
    colors = [color_map.get(cat, '#808080') for cat in plot_data.index]

    # 파이 차트 그리기
    wedges, texts, autotexts = axes[row, col].pie(plot_data, labels=plot_data.index, autopct='%1.1f%%',
                                                  startangle=90, colors=colors)

    # 레이블 텍스트 크기 조정
    plt.setp(texts, size=10)
    plt.setp(autotexts, size=10, weight="bold")

    axes[row, col].set_title(f'{age_label}', size=18)

plt.suptitle('연령대별 도서 장르 선호도', size=24)
plt.tight_layout()
plt.show()

# 5. 시간의 흐름에 따른 독서 분야 추세 (꺾은 선 그래프 (6x2) subplot(?))
 - x: 시간, y: Sales.sum(), elements: Category10

In [ ]:
category_top10 = filtered_df['Category'].value_counts().head(10).index

# 상위 10개 카테고리에 해당하는 데이터 필터링
date_sales_category = filtered_df[filtered_df['Category'].isin(category_top10)]

# Month 칼럼을 활용하여 6개월 단위로 집계
date_sales_category['Date'] = pd.to_datetime(date_sales_category['Date'])
date_sales_category = date_sales_category.set_index('Date').groupby('Category').resample('6M')['Sales'].sum().reset_index()

# 연도별 상위 10개 카테고리별 독서 구매량
plt.figure(figsize=(16, 8))
sns.lineplot(data=date_sales_category, x='Date', y='Sales', hue='Category', style='Category', markers=True, palette='Set2',
             linewidth=2)
plt.title('도서 판매량 Top.10 도서 트렌드 변화', fontsize=20)
plt.xlabel('기간', fontsize=16, loc='right')
plt.ylabel('독서량', fontsize=16, rotation=0, loc='top')
plt.legend(title='카테고리', title_fontsize=12, loc='upper right', fontsize=12)
plt.grid(False)

# 그래프 간 간격 조정
plt.tight_layout()
plt.show()

# 도서 트렌드 변화 확인
 - 시간 흐름에 따른 도서 종류의 트렌드 변화 추이를 확인할 수 있습니다.
  - 2019년도부터 2021년도 경제.경영 분야에 대한 도서 구매량이 폭증한 것을 확인할 수 있습니다. 이는 코로나 여파로 폭락했던 경제위기 상황을 타계하고 경기를 부양하기 위한 경제정책의 영향으로 주식에 대한 투자 열풍이 불기 시작하며 사회적인 관심이 경제분야 집중된 영향이 도서 출판 분야에도 영향을 미친 것으로 분석할 수 있습니다.

# 이후 개선하거나 업그레이드 가능한 항목

# 6. 결론

- 위 그래프를 통해 2017년도부터 현재까지 사람들의 관심분야(카테고리)가 어떻게 바뀌어가는지 확인이 가능했음.
- 아래는 '2017년 카테고리별 선호도'와 '2024년 카테고리별 선호도' 그래프로  
독자들의 관심분야가 17년도에 비해 어떻게 바뀌었는지 확인하고 가장 인기있는 카테고리 TOP3 에 한하여 추가 분석.

In [ ]:
# 2017년 카테고리별 선호도와 2024년 현재 카테고리별 선호도 2x1 subplot
import pandas as pd
import matplotlib.pyplot as plt

grouped_filtered_df = filtered_df.groupby(['Year', 'Category'])['Sales'].sum().unstack().fillna(0)
grouped_filtered_df = grouped_filtered_df[category_top10]

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Sales 크기에 따라 역순으로 정렬
sorted_grouped_filtered_df_2017 = grouped_filtered_df.loc[2017].sort_values(ascending=True)
sorted_grouped_filtered_df_2024 = grouped_filtered_df.loc[2024].sort_values(ascending=True)

categories = ['소설', '경제경영', '시/에세이', '인문', '자기계발', '유아동', '사회과학 / 역사', '가정과 생활', '만화/라이트노벨', '자연과 과학']

category_colors = {
    '소설': '#1f77b4',
    '경제경영': '#ff7f0e',
    '시/에세이': '#2ca02c',
    '인문': '#d62728',
    '자기계발': '#9467bd',
    '유아동': '#8c564b',
    '사회과학 / 역사': '#e377c2',
    '가정과 생활': '#7f7f7f',
    '만화/라이트노벨': '#bcbd22',
    '자연과 과학': '#17becf'
}

# 2017년 데이터 시각화
sorted_grouped_filtered_df_2017.plot(kind='barh', ax=axes[0], color=[category_colors[category] for category in sorted_grouped_filtered_df_2017.index])
axes[0].set_title('2017년 카테고리별 선호도')
axes[0].set_xlabel('Sales')
axes[0].set_ylabel('Category')

# 2024년 데이터 시각화
sorted_grouped_filtered_df_2024.plot(kind='barh', ax=axes[1], color=[category_colors[category] for category in sorted_grouped_filtered_df_2024.index])
axes[1].set_title('2024년 카테고리별 선호도')
axes[1].set_xlabel('Sales')
axes[1].set_ylabel('Category')

# 레이아웃 조정
plt.tight_layout()
plt.show()


- x축 값을 통해 전반적인 독서량이 현저히 줄은 것을 확인.
- 17년도에는 가장 많은 비중을 차지했던 '소설'분야가 현재는 선호도 7위에 위치한 것을 확인.
- 17년도에는 5위였던 '인문'분야가 24년도에는 1위까지 올라와 현재는 찾는 사람이 가장 많다는 것을 확인 가능.

In [ ]:
# 1등 카테고리 추출해서 책 타이틀 분석해서 (중복되는 단어?)왜 이게 인기있는지

- 17년도에 5위였던 '인문'분야가 24년도에 1위까지 올라온 것에 분석할 필요가 있다 생각하여 24년도 인문을 판매권수에 따라 내림차순으로 나타내봄.

In [ ]:
#  (filtered_df['Month']==5)
filtered_df[(filtered_df['Year']==2024) & (filtered_df['Category']=='인문')].sort_values(by='Sales', ascending=False)

In [ ]:
"인문 분야 책이름 추천해줘"

1. 2024년 현재 인기 카테고리 3분야 중 베스트셀러 TOP.10
 - 베스트셀러도서 선정 요인 분석.
 - 요소 : 가격,키워드,연령대별 평점, 페이지수, 베댓, 성별등

# 1. 연령대 별

총 3개 그래프 출력
- subplot(3x2)
- pie (size 조금 크게 ?)
- subplot(3x2)

In [ ]:
# 24년도 10, 20, 30, 40, 50, 60 카테고리 TOP3 (kind='barh')   ## 3행 2열

In [ ]:
# 독자 연령대 비율 groupby('Age')['Sales_Index'].sum()        ## pie chart

In [ ]:
# 24년도 10, 20, 30, 40, 50, 60 독자 남녀 비율 groupby(['Age', 'Gender'])['Sales_Index'].sum()    ## subplot piechart (x6) 3행 2열

# 2. 카테고리 별

총 3개 그래프 출력
- subplot(3x1)
- barh
- 꺾은선 그래프(size 너무크지 않게)

In [ ]:
# 출판일자 1월부터 12월 분포도 막대그래프 3개   ## 3행 1열 ( 인문 카테고리, 경제 경영 카테고리, 자기계발 카테고리 )

In [ ]:
# 인문 카테고리 연령대별 선호도 (kind='barh'),  인문 카테고리 독자 남녀 비율

In [ ]:
# 인문 카테고리 17년도 부터 지금까지 추세 (꺽은선 그래프) (인터파크 데이터셋 활용) figsize 너무 크지않게

# 3. 성별 별

총 2개 그래프 출력
- 수평 막대 그래프 (x: [10m, 20m, 30m, 40m, 50m, 60m]
- 수평 막대 그래프 (x: ['가정살림', '경제경영', ...]

In [ ]:
# 24년도 남자 독자 연령대 분포도 df[df['Gender']=='M'].groupby('Age')['Sales_Index'].sum() ( 막대 그래프 )

In [ ]:
# 24년도 남자 독자 카테고리 선호도 ( 10개 카테고리 막대그래프 sort할 필요 x )  ( 막대 그래프 )

# Final Output